# **LAB THREE: SELECTIVE EXCITATION**

This lab covers...

>-------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Setup Task: Run the Notebook**
> 
> 1. Edit the cell below to set the `LAB_USER_NAME` variable to your name
> 2. Click **Run->Run All Cells** in the in top menu bar of jupyterlab
> 3. Open the Table of Contents side-bar on the left edge of jupyterlab to aid in navigation
> 
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [6]:
LAB_USER_NAME = 'Sharon'

**Important**: To initialise this notebook, edit the cell above to set `LAB_USER_NAME` to your name, then click **Run->Run All Cells** in the top menu bar.

In [7]:
import panel as pn
pn.extension(raw_css=['''progress {margin: 0;}''']) # raw_css setting is a workaround for panel issue 4112
import sys
import os
import numpy as np

# add inline dashboard libraries to path so they can be imported later
sys.path.append('../../../dashboards-inline')

LAB_DIR = os.path.join('/home/data/', LAB_USER_NAME)
os.makedirs(LAB_DIR, exist_ok=True)
print('User data directory:', LAB_DIR)

User data directory: /home/data/Sharon


## 1. Projection Images
Thus far, the 2D image created in the labs have been projection images. These images are created using a rectangular RF-pulse (known as a hard pulse) and both frequency and phase encoding gradients (Fig.1,a). The frequency and phase encoding gradients allow the aquired signals to be sorted into 2D space to produce an image. The orientation of the image will depend on which axes that the frequency and phase gradients were applied along. Since there is no form of encoding in the third dimension of the sample, the signals along this axis will be summed in the 2D image.   

<center><img src="Images/projection2.0.png" width="1300"></center>
<center><figcaption style="width: 600px;">Figure 1: Setting the frequency and phase encoding in the x and z axes produces a projection imagine where the varying signal components in the y direction are summed into average values.  </figcaption></center>

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 1.1: Generate a Projection Image**
> 1. Insert the mystery sample (ilumr sample) at the correct depth.
> 2. Choose a the phase encoding and frequency encoding axes and start the experiment by pressing "Run". 
>
> **Question:** Can you read the text located in the centre of the phantom?
> 
> -------------------------------------------------------------------------------------------------------------------------------------------------------

TO DO: Insert a window that is 2D RARE without slice selection options.

As you can see, projection images aren't always useful because summing a large amount of signal in the object's third dimension makes it almost impossible to see details that are only present in specific parts of the sample. If we want to see detail in a specific portion of the object we need to be able to limit the amount of signal that is being summed in one direction. This is done through slice selection.

## 2. Slice Selection
In order to image a single slice rather than the entire sample, two main changes need to be made to the pulse sequence. 

1) A slice-select gradient must be added along the axis perpendicular to the plane of the desired slice. This causes the spins along the chosen axis to preccess at different frequencies making them frequency selectable. 

2) If we only want to excite a thin slice of the sample we will need a spacially selective RF-pulse. If the slice-select gradient were used with a hard pulse, all the spins would still be excited because a rectangular pulse contains the frequencies corresponding to all these spins' precession. To achieve a limited rectangular excitation profile in the frequency domain, we must apply a sinc shaped RF-pulse (known as a soft pulse) in the time domain (shown in Fig.2,a). 

Changing the parameters of the slice-select gradient and the sinc shaped soft pulse allows us to control the thickness of the slices and their position along the slice-select gradient axis.  

<center><img src="Images/slice2.0.png" width="1300"></center>
<center><figcaption style="width: 600px;">Figure 2: Slicing along the y axis allows the sample to be imaged at a specific cross section index     </figcaption></center>

## 3. Controlling Slice Thickness
Slices can not be infinitly thin because... Therefore, there will always be some level of summing and averaging along one axis when producing a 2D image. Choosing a slice thickness is a trade off between two main factors. A thicker slice will have more signal because there are more nuclear spins available for excitation. A thinner slice will have less averaging and therefore can show more detail. Adjusting thickness lets you find the optimal point for each application.

The thickness of the slice ($\Delta$z) is controlled by two main factors, the frequency range selected by the soft pulse ($\Delta$F) and the slice-select gradient ($G_{ss}$). This relationship is governed by the following equation:

$$\Delta z = \frac{\Delta F}{\gamma . G_{ss}} \tag{1}$$

where $\gamma$ is the gyromagnetic ratio. 

The following sections illustrate the relationship between RF-pulse shape, slice-select gradient strength, and the thickness of a slice.

### 3.1 Soft Pulse Shape relationship to slice thickness
The frequency range selected by the soft pulse ($\Delta$F) is determined the the shape of the soft pulse. The shape of the soft pulse also affects the shape of the excitation profile. While ideally this shape is a perfect rectangle that evenly excites a specific band of frequencies and does not excite others, this is often not the case. (explain curves and weird shapes and how it affects image quality). 

Soft pulses are designed in many ways. In their most basic form they are a mathematical sinc. 

<center><img src="Images/soft_pulse_parameters.png" width="800"></center>
<center><figcaption style="width: 300px;">Figure x:  </figcaption></center>


#### 3.1.1 Sinc Pulse
Theoretically, a perfect rectangle can be created in the frequency domain when an infinite sinc pulse is applied in the time domain. However, due to hardware limitations, it is not possible to implement this kind of sinc pulse. Instead, steps must be taken such as limiting the number of nodes and applying gaussian filters to the shape of the sinc (explain why these are done and what effect they have). 


> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 3.1.1.1: Visualising Sinc Pulse Design**
>The experiment below is designed to help you visualise how changing the parameters of a soft pulse changes the profile in the frequency domain.(explain what things cause issues e.g. ripples on the top of the spectrum and what this does to the final image).The Rf-Pulse you design will be used for the following experiments.
> 1. Experiment with the three soft pulse parameters and observe how the shape of the RF-pulse and slice profile changes. 
> 2. **Question:**
> 3. **Question:**
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [8]:
from matipo.util.pulseshape import calc_soft_pulse
from matipo.util.plots import SharedXPlot, ComplexPlot
from scipy.fft import fft,fftfreq, fftshift, ifftshift
from bokeh.plotting import figure

width_input = pn.widgets.FloatInput(name="width", start=1, end=2000, step=10, value=400, width=80)
n_lobe_input = pn.widgets.FloatInput(name="lobes", start=0, end=100, step=1, value=3, width=80)
apodization_input = pn.widgets.FloatInput(name="apodization", start=0, end=100, step=1, value=0, width=80)

p1 = ComplexPlot(
    title="Waveform",
    x_axis_label="Time (ms)",
    y_axis_label="Amplitude",
    height=400)

p2 = ComplexPlot(
    title="Spectrum",
    x_axis_label="Frequency (kHz)",
    y_axis_label="Spectral Density",
    height=400)

plot_row = pn.Row(p1.figure, p2.figure, sizing_mode='stretch_width')

global_shape = [1]
global_width = 100e-6

def update_plots(event):
    global global_width, global_shape 
    width = width_input.value*1e-6
    N = int(width / 1e-6)
    if N > 1000:
        N = 1000
    dt = width / N
    pts = np.sinc(np.linspace(-n_lobe_input.value, n_lobe_input.value,N)) 
    t = np.arange(N)*dt
    t_0 = (width-dt)/2
    pts*=np.exp(-apodization_input.value*(np.linspace(-1, 1, N)**2))
    global_shape = pts
    global_width = width
    freq = fftfreq(N,dt)
    spectrum = fft(pts)
    spectrum *= np.exp(-1j*np.pi*(width+dt)*freq) # fix phase of spectrum plot due to time offset
    freq = fftshift(freq)
    spectrum = fftshift(spectrum)
    p1.update_data(t, pts)
    p2.update_data(freq, spectrum)
    pn.io.push_notebook(plot_row)

# update plot when any value is changed
width_input.param.watch(update_plots, 'value')
n_lobe_input.param.watch(update_plots, 'value')
apodization_input.param.watch(update_plots, 'value')

# manually trigger for first update
width_input.param.trigger('value')

app = pn.Row(
    pn.Column(width_input, n_lobe_input, apodization_input),
    plot_row,
    sizing_mode='stretch_width')

app

Row(sizing_mode='stretch_width')
    [0] Column
        [0] FloatInput(end=2000, name='width', start=1, step=10, value=400, width=80)
        [1] FloatInput(end=100, name='lobes', start=0, step=1, value=3, width=80)
        [2] FloatInput(end=100, name='apodization', start=0, step=1, width=80)
    [1] Row(sizing_mode='stretch_width')
        [0] Bokeh(Figure)
        [1] Bokeh(Figure)

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 3.1.1.2: 1D Slice Profile**
> With the sinc pulse designed, we can now use it to create a 1D image of the slice profile. Later in the lab we will explore the effect the slice-select gradient has on the slice profile. For now, the gradient strength ($G_{ss}$) is fixed at 50% (add the actual value of this for calculations).    
> 1. Based on this value and the frequency range ($\Delta$F) of the sinc pulse you have designed, calculate the expected thickness of the slice profile.
> 2. Insert the shim sample (make sure to centre the sample using the depth gauge).
> 3. Run the experiment and check your answer against the results.
> 4. Change the shape of the RF-pulse and observe how this changes the slice profile. **Question:** How does the overall shape on the slice profile compare to the shape of the shape of the excitation profile.
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [9]:
from custom_pulse_SE_app import CustomPulseSEApp # from dashboards-inline directory that was added to sys.path

def estimate_a_90():
    shape_area = np.mean(global_shape)*global_width
    target_area = 0.3*32e-6
    return min(1, abs(target_area/shape_area))

# set some parameters directly
override_pars = dict(
    a_90=estimate_a_90,
    t_90=lambda: global_width,
    shape_90=lambda: global_shape,
    g_slice=(0,0,-0.5),
    g_read=(0,0,0.5),
    n_scans=2,
    n_samples=200,
    t_dw=10e-6, # using a long dwell time for narrow bandwith to more easily see the spectrum shape
    t_end=0.5
)

# create dashboard app
exp1_app = CustomPulseSEApp(
    override_pars=override_pars,
    show_magnitude=True,
    show_complex=True,
    enable_run_loop=True,
    flat_filter = True
)

exp1_app.plot1.figure.height=400
exp1_app.plot2.figure.height=400

# display layout
pn.Column(
    exp1_app.main(),
    sizing_mode='stretch_width'
)

Column(sizing_mode='stretch_width')
    [0] Column(sizing_mode='stretch_both')
        [0] Row(sizing_mode='stretch_both')
            [0] Bokeh(Figure)
            [1] Bokeh(Figure)
        [1] Row(sizing_mode='stretch_width')
            [0] Button(align='end', button_type='success', name='Run', sizing_mode='stretch_width')
            [1] Button(align='end', button_type='success', name='Run Loop', sizing_mode='stretch_width')
            [2] Button(align='end', button_type='danger', name='Abort', sizing_mode='stretch_width')
            [3] Progress(height=31, sizing_mode='stretch_width', value=0)
            [4] TextInput(align='end', disabled=True, sizing_mode='stretch_width', value='Idle')

TO DO: add a part explaining the prephasing gradient 

Notice the twisting in the imaginery components of the spectrum. This is due to dephasing. The application of the slice-select gradient during the soft pulse causes the spins to accumulate different amounts of phase based on their position along the the gradient's axis. This phase dispersion of the transverse magnetization will result in a loss of signal. Since the dephasing that occurs is linear, the phase effects can be cancelled by applying a gradient in the opposite direction. This is done using a rephasing gradient. In general, the area of the rephasing gradient is half that of the slice-select gradient. This is done under the assumption that the majority of the spins are tipped into the transverse plane at the centre of the 90-degree pulse and, therefore, phase dispersion primarily occurs during the second half of the slice select gradient. While this assumption is a good starting point, the size of the refocusing gradient needs to be calculated based on the soft pulse design if the phase effects are to be cancelled perfectly.

<center><img src="Images/refocusing_gradient.png" width="1300"></center>
<center><figcaption style="width: 300px;">Figure x:  </figcaption></center>

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 3.1.1.3: Refocusing Gradient**
> 1. Run the experiment.
> 2. Select the refocusing option and run again. Notice how the imaginary component of the spectrum changes. 
> 3. The rephasing gradient for this experiment is half the area of the slice-select gradient. Notice how this doesn't perfectly cancel all the phase effects. Try altering the sinc pulse design in **Task 3.1.1.1** then run **Task 3.1.1.3** again. See how increasing the duration of the sinc pulse effects how well the 50% refocusing gradient performs.
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [10]:
from custom_pulse_SE_refocusing_app import CustomPulseSEApp # from dashboards-inline directory that was added to sys.path

input_refocusing = pn.widgets.Checkbox(name='Refocusing')

def estimate_a_90():
    shape_area = np.mean(global_shape)*global_width
    target_area = 0.3*32e-6
    return min(1, abs(target_area/shape_area))

# set some parameters directly
override_pars = dict(
    a_90=estimate_a_90,
    t_90=lambda: global_width,
    shape_90=lambda: global_shape,
    g_slice=(0,0,-0.5),
    g_read=(0,0,0.5),
    n_scans=2,
    n_samples=200,
    t_dw=10e-6, # using a long dwell time for narrow bandwith to more easily see the spectrum shape
    t_end=0.5, 
    enable_refocusing = input_refocusing
)

# create dashboard app
exp2_app = CustomPulseSEApp(
    override_pars=override_pars,
    show_magnitude=True,
    show_complex=True,
    enable_run_loop=True,
    flat_filter = True
)

exp2_app.plot1.figure.height=400
exp2_app.plot2.figure.height=400

# display layout
pn.Column(
    input_refocusing,
    exp2_app.main(),
    sizing_mode='stretch_width'
)

Column(sizing_mode='stretch_width')
    [0] Checkbox(name='Refocusing')
    [1] Column(sizing_mode='stretch_both')
        [0] Row(sizing_mode='stretch_both')
            [0] Bokeh(Figure)
            [1] Bokeh(Figure)
        [1] Row(sizing_mode='stretch_width')
            [0] Button(align='end', button_type='success', name='Run', sizing_mode='stretch_width')
            [1] Button(align='end', button_type='success', name='Run Loop', sizing_mode='stretch_width')
            [2] Button(align='end', button_type='danger', name='Abort', sizing_mode='stretch_width')
            [3] Progress(height=31, sizing_mode='stretch_width', value=0)
            [4] TextInput(align='end', disabled=True, sizing_mode='stretch_width', value='Idle')

TO DO: 
- Talk about non linearity and why it matters - does this relate to bloch equations?
- fix the calibration of the amplitude in the experiment 

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 3.1.1.3: Non Linearity of Sinc Pulse**
> 1. Insert the shim sample
> 2. Run the experiment 
> 3. Change the amplitude of the soft pulse. Notice how the spectrum changes. **Question:** What happens to the spectrum once you reach a flip angle of 180-degrees.
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [11]:
from custom_pulse_SE_app import CustomPulseSEApp # from dashboards-inline directory that was added to sys.path

amplitude_input = pn.widgets.FloatInput(name="Amplitude", start=1, end=360, step=10, value=90, width=80)

def estimate_a_90():
    shape_area = np.mean(global_shape)*global_width
    target_area = 0.3*32e-6
    return min(1, abs(target_area/shape_area))

# set some parameters directly
override_pars = dict(
    a_90=lambda: (amplitude_input.value/90)*estimate_a_90(),
    t_90=lambda: global_width,
    shape_90=lambda: global_shape,
    g_slice=(0,0,-0.5),
    g_read=(0,0,0.5),
    n_scans=2,
    n_samples=200,
    t_dw=10e-6, # using a long dwell time for narrow bandwith to more easily see the spectrum shape
    t_end=0.5
)

# create dashboard app
exp3_app = CustomPulseSEApp(
    override_pars=override_pars,
    show_magnitude=True,
    show_complex=True,
    enable_run_loop=True,
    flat_filter = True
)

exp3_app.plot1.figure.height=400
exp3_app.plot2.figure.height=400

# display layout
pn.Column(
    amplitude_input,
    exp3_app.main(),
    sizing_mode='stretch_width'
)

Column(sizing_mode='stretch_width')
    [0] FloatInput(end=360, name='Amplitude', start=1, step=10, value=90, width=80)
    [1] Column(sizing_mode='stretch_both')
        [0] Row(sizing_mode='stretch_both')
            [0] Bokeh(Figure)
            [1] Bokeh(Figure)
        [1] Row(sizing_mode='stretch_width')
            [0] Button(align='end', button_type='success', name='Run', sizing_mode='stretch_width')
            [1] Button(align='end', button_type='success', name='Run Loop', sizing_mode='stretch_width')
            [2] Button(align='end', button_type='danger', name='Abort', sizing_mode='stretch_width')
            [3] Progress(height=31, sizing_mode='stretch_width', value=0)
            [4] TextInput(align='end', disabled=True, sizing_mode='stretch_width', value='Idle')

#### 3.1.2 SLR Pulse Design
TO DO: Explain why based on the last exercises that optimised pulses are better than sinc pulses for what we're doing. Talk about the differences between them. 


> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 3.1.2.1: SLR Pulse Calibration**
> TO DO: Insert a window that runs a pulse sequence and plots the slice profile. Amplitude of the RF pulse can be altered to show how the shape of the slice profile changes based on the amplitude of the pulse. 
> 1. Use
> -------------------------------------------------------------------------------------------------------------------------------------------------------

### 3.2 Slice Select Gradient relationship to slice thickness

The thickness of the slice that is selected is also affected by the strength of the gradient. 

Fig.x illustrates how three identical RF-pulses can result in different slice thicknesses when the gradients are changed.

<center><img src="Images/slice_thickness.png" width="1400"></center>
<center><figcaption style="width: 300px;">Figure x:  </figcaption></center>

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 3.2.1: ???**
> TO DO: Insert 2D RARE that is set to show just the width of the slice. Have adjustable bandwidth and gradient parameters so they can see the slice width change as the parameters are changed.
> Relate this part to the actual units - create a desired thickness in mm based on gradient and bandwidth
> 1. Change the...
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In many of ilumr's dashboards you can select the desired slice thickness and the bandwidth and gradient will both be automatically adjusted to allow for this thickness. This is done by... 

## 4. Controlling Slice Position
TO DO: Explain centre frequency and how this is controlled. Maybe an equation 

$$F_{c} = \gamma(B_0 + z.G_{ss})$$

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 4.2: Locating the Hidden Text**
> TO DO: insert a standard 2D RARE dashboard where they can play around with slice selection in different axes to find the ilumr text in the ilumr phantom
> 1. Change the...
> -------------------------------------------------------------------------------------------------------------------------------------------------------

Things to add:
- Need to add a section talking about center frequency for the slice selection pulse and how this allows you to slice at different points in the sample
- Changing the direction/axis of slice selection
- Non-linearity of Bloch equations (talk about distortion in slice profile once go past a certain point)
- How the pulse calibration dashboard works 
- Calibration methods
- Tying everything to physical measurements - how does the actual gradient strength and B1 relate to physical slice thickness. How to calculate this.